### Markdown example
Blab bla documentation


In [3]:
import requests
import untangle
import pprint

pp = pprint.PrettyPrinter(indent=4)

r = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403', auth=('user', 'pass'))
print r
print ''
print 'Status code: ' 
print r.status_code
print ''
print 'Headers: '
print r.headers
print ''


xmlObj = untangle.parse(r.text)

pmid_lst = []

for ref in xmlObj.pmc_articleset.article.back.ref_list.ref:
    this_cite = ref.citation
    
    if hasattr(this_cite, 'pub_id'):
        pubs = this_cite.pub_id
        if type(pubs) == list:
            for pub_el in pubs:
                if pub_el['pub-id-type'] == 'pmid':
                    #print 'Reference: '
                    #print this_cite.article_title.cdata
                    #print 'Entry type:'
                    #print type(pub_el)
                    #print 'Has PMID:'
                    #print pub_el.cdata
                    pmid_lst.append(pub_el.cdata)
        elif pubs['pub-id-type'] == 'pmid':
            #print 'Reference: '
            #print this_cite.article_title.cdata
            #print 'Entry type:'
            #print type(pubs)
            #print 'Has PMID:'
            #print pubs.cdata
            pmid_lst.append(pubs.cdata)
            
print 'All cited PMIDs:'
print pmid_lst
print len(pmid_lst)

<Response [200]>

Status code: 
200

Headers: 
{'X-XSS-Protection': '1; mode=block', 'NCBI-PHID': '03ED5B8C816250710000000003D303C0', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'ncbi_sid=03ED5B8C81630B51_0979SID; domain=.nih.gov; path=/; expires=Mon, 30 Oct 2017 17:41:09 GMT', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Vary': 'Accept-Encoding', 'Keep-Alive': 'timeout=1, max=10', 'Server': 'Apache', 'Content-Security-Policy-Report-Only': "default-src https:; script-src https: 'unsafe-inline' 'unsafe-eval'; font-src https: data:; img-src https: data:; style-src https: 'unsafe-inline'; report-uri https://www.ncbi.nlm.nih.gov/corecgi/csp/csp.cgi", 'NCBI-SID': '03ED5B8C81630B51_0979SID', 'Connection': 'Keep-Alive', 'X-UA-Compatible': 'IE=Edge', 'Cache-Control': 'private', 'Date': 'Sun, 30 Oct 2016 17:41:09 GMT', 'Access-Control-Allow-Origin': '*', 'Content-Type': 'text/x

In [3]:
import sys
sys.path.append('../web/lib')
from ConnectEutils import ConnectEutils
import pandas as pd


eutils = ConnectEutils()

pd.DataFrame(eutils.get_cited_PMID('212403'), columns=['PMID'])

ImportError: No module named pandas

In [6]:
import xml.etree.ElementTree as ET
import requests 
import urllib2

tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403' ))
tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=4807304' ))


pmid_lst = []

for element in tree.iter():
    if element.tag == 'pub-id':
        if element.attrib['pub-id-type'] == 'pmid':
            pmid_lst.append(element.text)
            
print pmid_lst


[]


In [13]:
import sys
sys.path.append('../web/lib')
from ConnectEutils import ConnectEutils


eutils = ConnectEutils()

pub1 = eutils.get_cited_PMID_etree(PMID="5055858")
pub2 = eutils.get_cited_PMID_etree(PMID="5107590")

#print pub1
#print pub2

print list(set(pub1) & set(pub2))

[]


In [3]:
# Simulate a browser session
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph

graphsession = GraphSession('5055858,5107590,5063561')
graphsession.parse_input()
graphsession.load_citations()
resultgraph = ResultGraph()
print graphsession.cite_dict
resultgraph.populate_from_cite_dict(graphsession.cite_dict)

this_graph = resultgraph.G

for n in this_graph:
    print n
data = json_graph.node_link_data(this_graph)
s = json.dumps(data)
print s
#resultgraph.get_json()

{'5063561': None, '5107590': None, '5055858': None}


TypeError: 'NoneType' object is not iterable

In [4]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph

graphsession = GraphSession('24752654')
graphsession.parse_input()
graphsession.load_citations()
resultgraph = ResultGraph()
print graphsession.cite_dict
resultgraph.populate_from_cite_dict(graphsession.cite_dict)

this_graph = resultgraph.G

for n in this_graph:
    print n
data = json_graph.node_link_data(this_graph)
s = json.dumps(data)
print s
#resultgraph.get_json()
#27729734,27895396,27785449

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=24752654&tool=GraphSearch
{'24752654': ['24752654', '24091329', '23746840', '23420871', '23386393', '23336100', '22918226', '22865453', '22777646', '22722893', '22635005', '22357162', '22318521', '22258609', '22216980', '21989406', '21876726', '21653254', '21640565', '21376230', '20884633', '20631206', '20628381', '20550684', '20179207', '20124221', '20019050', '20016485', '19603039', '19460998', '19381331', '19131956', '19033363', '18796163', '18775299', '18613948', '18589210', '18573355', '17656355', '17504872', '17486113', '17255514', '17109012', '16890165', '16651434', '16453000', '16102175', '14729627', '14679214', '13298683', '12716131', '12118079', '11872830', '11390481', '11248153', '10722846', '9563066', '8349690', '1905840', '1013510', '212572']}
1013510
20550684
22918226
22865453
18775299
12118079
20628381
19033363
8349690
11390481
14729627
18796163
17656355
16453000
2001648

## Try converting PMC ID to pmid

In [2]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph

graphsession = GraphSession('24752654')
graphsession.parse_input()
graphsession.load_citations()
resultgraph = ResultGraph()
print graphsession.cite_dict
resultgraph.populate_from_cite_dict(graphsession.cite_dict)


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=24752654&tool=GraphSearch
eLinkResult
LinkSet
DbFrom
IdList
Id
24752654
LinkSetDb
DbTo
LinkName
Link
Id
24091329
Link
Id
23746840
Link
Id
23420871
Link
Id
23386393
Link
Id
23336100
Link
Id
22918226
Link
Id
22865453
Link
Id
22777646
Link
Id
22722893
Link
Id
22635005
Link
Id
22357162
Link
Id
22318521
Link
Id
22258609
Link
Id
22216980
Link
Id
21989406
Link
Id
21876726
Link
Id
21653254
Link
Id
21640565
Link
Id
21376230
Link
Id
20884633
Link
Id
20631206
Link
Id
20628381
Link
Id
20550684
Link
Id
20179207
Link
Id
20124221
Link
Id
20019050
Link
Id
20016485
Link
Id
19603039
Link
Id
19460998
Link
Id
19381331
Link
Id
19131956
Link
Id
19033363
Link
Id
18796163
Link
Id
18775299
Link
Id
18613948
Link
Id
18589210
Link
Id
18573355
Link
Id
17656355
Link
Id
17504872
Link
Id
17486113
Link
Id
17255514
Link
Id
17109012
Link
Id
16890165
Link
Id
16651434
Link
Id
16453000
Link
Id
16102175
Link
Id
14729627
Lin

In [2]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from ConnectEutils import ConnectEutils

eutils = ConnectEutils()

PMC_ID = 'PMC4413964'

pmid = eutils.get_pmid_from_PMC(PMC_ID)
print pmid

cite_lst = eutils.get_cited_pub(PMC_ID)
print cite_lst

25918260
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=25918260&tool=GraphSearch
['25918260', '24606128', '24576654', '24520944', '24415877', '24384685', '24350564', '24325272', '24293372', '24283303', '24255719', '24251922', '24232581', '24210956', '24172183', '24171787', '24155775', '24123097', '24117517', '24083739', '24081937', '24064753', '24056623', '24002499', '23982274', '23980084', '23980077', '23962244', '23945731', '23922471', '23884750', '23869689', '23840128', '23800278', '23796082', '23741309', '23737955', '23719667', '23660416', '23597497', '23565616', '23561638', '23547075', '23523578', '23483664', '23466316', '23456227', '23447460', '23426336', '23402625', '23400333', '23377952', '23375519', '23331661', '23326137', '23288584', '23269606', '23216217', '23213040', '23188531', '23178708', '23157398', '23145027', '23134976', '23099728', '23095613', '23079897', '23023009', '23001077', '22994906', '22965574', '22948836'

In [5]:
x = 'PMC4413964'
if 'PMC' in x:
    print 'yup'

yup
